<a href="https://colab.research.google.com/github/thesamasaurus/m19-ml-bot/blob/main/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data prep for deeplearning on MTG draftsim drafts

## Importing modules

In [ ]:
import os
import re
import pandas as pd
import copy
import numpy as np
import pickle

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Importing list of all card names

#### card names are split between two files, lands and spells, we need to import them both

In [ ]:
data_folder = "/content/gdrive/MyDrive/Colab Notebooks/m19_drafts/m19_drafts"
display(os.listdir(data_folder))

['m19_land_rating.tsv',
 'm19_rating.tsv',
 'LICENSE.txt',
 'README.txt',
 'test.csv',
 'train.csv']

In [ ]:
cur_set = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/m19_drafts/m19_drafts/m19_rating.tsv", delimiter="\t")

In [ ]:
cur_set.head()

,Name,Casting Cost 1,Casting Cost 2,Card Type,Rarity,Rating
0,Crucible_of_Worlds,3,none,Spell,M,1.5
1,"Nicol_Bolas,_the_Ravager",1UBR,none,Creature,M,4.6
2,Scapeshift,2GG,none,Spell,M,1.5
3,"Sarkhan,_Fireblood",1RR,none,Planeswalker,M,3.3
4,Resplendent_Angel,1WW,none,Creature,M,4.3


In [ ]:
cur_set_lands = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/m19_drafts/m19_drafts/m19_land_rating.tsv", delimiter="\t")

In [ ]:
cur_set_lands.head()

,Name,Casting Cost 1,Casting Cost 2,Card Type,Rarity,Rating
0,Plains,0,W,Land,C,-1.0
1,Plains_1,0,W,Land,C,-1.0
2,Plains_2,0,W,Land,C,-1.0
3,Plains_3,0,W,Land,C,-1.0
4,Plains_4,0,W,Land,C,-1.0


## Defining hash table of contents for onehot encoding

In [ ]:
all_cards_array = []
for i in cur_set_lands["Name"]:
    all_cards_array.append(i)
for j in cur_set["Name"]:
    all_cards_array.append(j)
len(all_cards_array)

285

In [ ]:
card_index = {}
for i in range(0, len(all_cards_array)):
    card_index[all_cards_array[i]] = i

In [ ]:
card_index_inverted = {v: k for k, v in card_index.items()} #inverting the dictionary 

## Importing Draft Data

In [ ]:
drafts_train = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/m19_drafts/m19_drafts/train.csv", delimiter=',', header=1)

In [ ]:
drafts_test = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/m19_drafts/m19_drafts/test.csv")

In [ ]:
print(len(drafts_train), len(drafts_test))

86357 21589


In [ ]:
drafts_train.columns = ["draftid", "set", "player_1", "player_2", "player_3", "player_4", "player_5", "player_6", "player_7", "player_8"]

In [ ]:
drafts_test.columns = ["draftid", "set", "player_1", "player_2", "player_3", "player_4", "player_5", "player_6", "player_7", "player_8"]

In [ ]:
drafts_test.head()

,draftid,set,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8
0,1312928,M19,"Lena,_Selfless_Champion,Luminous_Bonds,Shock,H...","Dismissive_Pyromancer,Luminous_Bonds,Dragon_Eg...","Rabid_Bite,Declare_Dominance,Declare_Dominance...","Star-Crowned_Stag,Herald_of_Faith,Surge_Mare,G...","Lena,_Selfless_Champion,Salvager_of_Secrets,An...","Open_the_Graves,Luminous_Bonds,Skymarch_Bloodl...","Palladia-Mors,_the_Ruiner,Vigilant_Baloth,Decl...","Vivien's_Invocation,Rabid_Bite,Gallant_Cavalry..."
1,1222597,M19,"Open_the_Graves,Fell_Specter,Fell_Specter,Hire...","Dwindle,Gargoyle_Sentinel,Star-Crowned_Stag,St...","Luminous_Bonds,Angel_of_the_Dawn,Arcane_Encycl...","Lathliss,_Dragon_Queen,Fiery_Finish,Arcane_Enc...","Hieromancer's_Cage,Lich's_Caress,Thorn_Lieuten...","Mystic_Archaeologist,Rabid_Bite,Declare_Domina...","Bone_Dragon,Strangling_Spores,Epicure_of_Blood...","Nicol_Bolas,_the_Ravager,Lich's_Caress,Dwindle..."
2,1260654,M19,"Lightning_Strike,Elvish_Rejuvenator,Shock,Cent...","Demon_of_Catastrophes,Patient_Rebuilding,Star-...","Phylactery_Lich,Vine_Mare,Plague_Mare,Volcanic...","Rabid_Bite,Bristling_Boar,Exclusion_Mage,Gargo...","Mentor_of_the_Meek,Volcanic_Dragon,Havoc_Devil...","Goreclaw,_Terror_of_Qal_Sisma,Rabid_Bite,Mirro...","Lich's_Caress,Surge_Mare,Knightly_Valor,Skymar...","Plague_Mare,Herald_of_Faith,Gallant_Cavalry,Su..."
3,1307575,M19,"Dismissive_Pyromancer,Strangling_Spores,Graved...","Demon_of_Catastrophes,Sift,Skymarch_Bloodlette...","Graveyard_Marshal,Dryad_Greenseeker,Dwindle,Gh...","Death_Baron,Lich's_Caress,Horizon_Scholar,Chil...","Sift,Star-Crowned_Stag,Vine_Mare,Centaur_Cours...","Prodigious_Growth,Sleep,Skilled_Animator,Lumin...","Dismissive_Pyromancer,Vampire_Sovereign,Gallan...","Dark-Dweller_Oracle,Shock,Ajani's_Pridemate,To..."
4,1222964,M19,"Lightning_Strike,Ajani's_Pridemate,Angel_of_th...","Sigiled_Sword_of_Valeron,Vivien's_Invocation,S...","Goblin_Trashmaster,Luminous_Bonds,Fell_Specter...","Patient_Rebuilding,Sift,Departed_Deckhand,Gear...","Switcheroo,Knightly_Valor,Gallant_Cavalry,Pega...","Lightning_Strike,Strangling_Spores,Strangling_...","Lightning_Strike,Runic_Armasaur,Gift_of_Paradi...","Electrify,Switcheroo,Surge_Mare,Skyrider_Patro..."


### Note:  Human Drafter is player_1

## Defining a function to find string representation of onehot vector for use later

In [ ]:
def find_contense(one_hot):
    pack = []
    for i in range(0, len(one_hot)):
        if (one_hot[i] == 0):
            pass
        if (one_hot[i] > 0):
            for j in range(0, one_hot[i]):
                pack.append(card_index_inverted[i])
    return pack

In [ ]:
example = find_contense([1, 0, 0, 0, 0, 1])
example

['Plains', 'Island']

## Translating Data into Onehot Encoding

### Defining a list of player names

In [ ]:
col_str = "player_"
list_of_players = []
for i in range(1, 9):
    list_of_players.append(col_str + str(i))

In [ ]:
list_of_players #list of player names, player 1 is hero

['player_1',
 'player_2',
 'player_3',
 'player_4',
 'player_5',
 'player_6',
 'player_7',
 'player_8']

### Define function to take a row of the dataset, and turn into a list of len 8

In [ ]:
def split_draft_text_by_player(dataset, list_of_players, draft):
    player_split_draft = []
    for i in range(0, len(list_of_players)):
        player_split_draft.append(dataset[list_of_players[i]][draft])
    return player_split_draft

In [ ]:
player_split_draft = split_draft_text_by_player(drafts_test, list_of_players, 1)

In [ ]:
player_split_draft[0]

"Open_the_Graves,Fell_Specter,Fell_Specter,Hired_Blade,Macabre_Waltz,Fire_Elemental,Hired_Blade,Viashino_Pyromancer,Rustwing_Falcon,Macabre_Waltz,Manalith,Tolarian_Scholar,Crash_Through,Crash_Through,Ajani's_Welcome,Death_Baron,Electrify,Boggart_Brute,Dragon_Egg,Infectious_Horror,Abnormal_Endurance,Fire_Elemental,Child_of_Night,Doomed_Dissenter,Havoc_Devils,Duress,Two-Headed_Zombie,Naturalize,Lava_Axe,Forest_4,Sarkhan,_Fireblood,Strangling_Spores,Brawl-Bash_Ogre,Brawl-Bash_Ogre,Shock,Goblin_Instigator,Trumpet_Blast,Alpine_Moon,Viashino_Pyromancer,Viashino_Pyromancer,Hired_Blade,Abnormal_Endurance,Mighty_Leap,Wall_of_Vines,Island_3"

### Using a Regex to split the block of text into a list of picks.  Output dims 8x45

In [ ]:
def regex_to_list_picks(entire_draft):
    seperated_pools = []
    for i in entire_draft:
        temp = re.split(",(?=[A-Z])", i)
        seperated_pools.append(temp)
    return seperated_pools

In [ ]:
seperated_pools = regex_to_list_picks(player_split_draft)

In [ ]:
seperated_pools[0][:5] #seperated_pools[player 1's][first 5 picks]

['Open_the_Graves',
 'Fell_Specter',
 'Fell_Specter',
 'Hired_Blade',
 'Macabre_Waltz']

### Splitting each players 45 picks into three packs of 15.  Output dims 8x3x15

In [ ]:
def seperate_pool_into_packs(each_players_pool):
    pack_seperated_draft = []
    for player in range(0, 8):
        player_picks = []
        for i in range(1,4):
            if (i == 1):
                pack = []
                for j in range(0, 15):
                    pack.append(each_players_pool[player][j])
                player_picks.append(pack)
            if (i == 2):
                pack = []
                for j in range(15, 30):
                    pack.append(each_players_pool[player][j])
                player_picks.append(pack)
            if (i == 3):
                pack = []
                for j in range(30, 45):
                    pack.append(each_players_pool[player][j])
                player_picks.append(pack)
        pack_seperated_draft.append(player_picks)
    return pack_seperated_draft

In [ ]:
pack_seperated_draft = seperate_pool_into_packs(seperated_pools)

In [ ]:
pack_seperated_draft[4][2] #player 5's 3rd pack

["Hieromancer's_Cage",
 'Pegasus_Courser',
 "Lich's_Caress",
 "Lich's_Caress",
 'Macabre_Waltz',
 'Abnormal_Endurance',
 'Skyscanner',
 'Two-Headed_Zombie',
 'Make_a_Stand',
 'Satyr_Enchanter',
 'Dwarven_Priest',
 'Cinder_Barrens',
 "Knight's_Pledge",
 'Smelt',
 'Omniscience']

### Turning this list into a layered dictionary for easier lookup, generating onehot of the pack opened

In [ ]:
def generate_packs_opened_dict(packs_opened_array):
    pack_dict = {}
    pack_dict_onehot = {}
    for players in range(0,8):
        player_packs = {}
        player_packs_onehot = {}
        for packs in range(0,3):
            pack = []
            onehot = np.zeros(285, dtype=np.int8)
            for picks in range(0,15):
                index = (players+ picks) % 8
                pick = packs_opened_array[index][packs][picks]
                pack.append(packs_opened_array[index][packs][picks])
                onehot[card_index[pick]] = 1
            player_packs["pack_" + str(packs + 1)] = pack
            player_packs_onehot["pack_" + str(packs + 1)] = onehot
        pack_dict["player_"+ str(players+1)] = player_packs
        pack_dict_onehot["player_"+ str(players+1)] = player_packs_onehot
    return pack_dict, pack_dict_onehot

In [ ]:
pack_dict, pack_dict_onehot = generate_packs_opened_dict(pack_seperated_draft)

In [ ]:
pack_dict_onehot["player_1"]["pack_2"]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [ ]:
np.count_nonzero(pack_dict_onehot["player_1"]["pack_1"] == 1) #as expected, 15 values give coded as 1 in pack

15

### This is a function that reshapes the data to make a following step easier

In [ ]:
def generate_feeder_array(pack_dictionary):  #this function just reshapes the data for easier use in next function
    feeder_array = []
    for i in range(0,8):
        for j in range(0,3):
            feeder_array.append(pack_dictionary["player_" + str(i+1)]["pack_" + str(j+1)])
    return feeder_array

In [ ]:
#feeder_array is a list with 24 entries.  1-3 are player 1's packs(1,2,3), 4-6 player twos packs(1,2,3), etc 

In [ ]:
feeder_array = generate_feeder_array(pack_dict)

In [ ]:
feeder_array[0]

['Open_the_Graves',
 'Gargoyle_Sentinel',
 'Arcane_Encyclopedia',
 'Goblin_Instigator',
 'Skymarch_Bloodletter',
 'Gift_of_Paradise',
 'Walking_Corpse',
 'Smelt',
 'Rustwing_Falcon',
 'Mighty_Leap',
 'Knight_of_the_Tusk',
 'Titanic_Growth',
 'Rhox_Oracle',
 'Highland_Game',
 'Timber_Gorge']

### Generating the packs seen by each player in each pick in string and onehot form

To recreate each pick seen by each player in the draft, we must take starting pack of each player, remove their pick, and then shift the pack in the correct direction, and repeat the process.  To keep track of the pack each player has at any given point, we use a list named pack_array, [0,1,2,3,4,5,6,7].  0 is the pack opened by player 0.  Every time a pick is made by all players, the last value of the array is popped out and shifted to the front.  We use the formula that considers pick, player number, and the current pack_array[player] to determine the contense of the pack, provided by feeder_array, at any given point.  Pack_array is nested within pack loop, because after each pack is finished, players start the next pack by opening their own pack.  We start by using a deepcopy because we don't want to destroy the orig packs in this process.

In [ ]:
def create_all_picks_seen(packs_opened, pack_seperated_draft, seperated_pools, target_player):
    packs_shown_player = {} #dict we will store all picks for all players
    packs_shown_player_onehot = {}
    pool_at_pick = {} #pool of player at that point
    pool_at_pick_onehot = {}
    pick_onehot = {}
    counter = 0 #counts picks so far in draft
    modified_packs = copy.deepcopy(packs_opened)  #we make a deepcopy to mutate the packs and retain orig
    
    for pack in range(0,3):
        pack_array = [0,1,2,3,4,5,6,7] #we pop the end and put it at the beginning for each new pick, reset after pack
        for pick in range(0,15):
            for player in range(0, 8):
#                 print("pack_seen:", pack_array[player], "player:", player, "pack", pack, "pick", pick)
                pick_made = pack_seperated_draft[player][pack][pick]
#                 print("pick: "+ pick_made)
#                 print("pack seen: ", modified_packs[pack_array[player]*3 + pack])
                pool = seperated_pools[player][:counter]
                pick_alternate_method = seperated_pools[player][counter]
        
                if (player == target_player):
                
                    pick_made_onehot = np.zeros(285, dtype=np.int8)
                    pick_made_onehot[card_index[pick_alternate_method]] = 1
                    
                    pick_onehot["player_" + str(player+1) + "_pack_" + str(pack+1) + "_pick_" + str(pick+1)] = pick_made_onehot
                    packs_shown_player["player_" + str(player+1) + "_pack_" + str(pack+1) + "_pick_" + str(pick+1)] = copy.deepcopy(modified_packs[pack_array[player]*3 + pack])
                    
                    active_pack = copy.deepcopy(modified_packs[pack_array[player]*3 + pack])
                    onehot_packs = np.zeros(285, dtype = np.int8)
                    
                    for k in active_pack:
                        onehot_packs[card_index[k]] = 1
                    packs_shown_player_onehot["player_" + str(player+1) + "_pack_" + str(pack+1) + "_pick_" + str(pick+1)] = onehot_packs
                   
                    pool_at_pick["player_" + str(player+1) + "_pack_" + str(pack+1) + "_pick_" + str(pick+1)] = pool
                    
                    #saving the onehot of the pool at the point before the pick was made
                    onehot_picks = np.zeros(285, dtype = np.int8)
                    
                    for j in pool:
                        onehot_picks[card_index[j]] = onehot_picks[card_index[j]] + 1
                    pool_at_pick_onehot["player_" + str(player+1) + "_pack_" + str(pack+1) + "_pick_" + str(pick+1)] = onehot_picks
                
                modified_packs[pack_array[player]*3 + pack].remove(pick_made)
                
            counter = counter + 1
            popout = pack_array.pop(-1)
            pack_array.insert(0, popout)
            
    return packs_shown_player, pool_at_pick, packs_shown_player_onehot, pool_at_pick_onehot, pick_onehot

In [ ]:
packs_shown_player, pool_at_pick, packs_shown_player_onehot, pool_at_pick_onehot, pick_onehot  = create_all_picks_seen(feeder_array, pack_seperated_draft, seperated_pools, 0)

### Example output

In [ ]:
pool_at_pick_onehot["player_1_pack_1_pick_11"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [ ]:
packs_shown_player["player_1_pack_2_pick_12"]

['Two-Headed_Zombie', 'Wall_of_Mist', 'Mighty_Leap', 'Thud']

In [ ]:
pool_at_pick["player_1_pack_1_pick_9"]

['Open_the_Graves',
 'Fell_Specter',
 'Fell_Specter',
 'Hired_Blade',
 'Macabre_Waltz',
 'Fire_Elemental',
 'Hired_Blade',
 'Viashino_Pyromancer']

### Making a smaller training set for faster errorproofing of ML

In [ ]:
drafts_train_start = 30000 #use this variable in conjunction with the export var line below to define train size
drafts_test_start = 3000 #use this variable in conjunction with the export var line below to define train size

In [ ]:
drafts_train_export = copy.deepcopy(drafts_train)
drafts_test_export = copy.deepcopy(drafts_test)

In [ ]:
drafts_train_export = drafts_train_export[drafts_train_start:60000]
drafts_test_export = drafts_test_export[drafts_test_start:6000]

In [ ]:
drafts_train_export.head()

,draftid,set,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8
30000,1282610,M19,"Plague_Mare,Regal_Bloodlord,Ajani's_Pridemate,...","Bone_Dragon,Star-Crowned_Stag,Lich's_Caress,Ma...","Vine_Mare,Lightning_Mare,Draconic_Disciple,Sho...","Arcades,_the_Strategist,Essence_Scatter,Aerial...","Sai,_Master_Thopterist,Angel_of_the_Dawn,Total...","Lich's_Caress,Shock,Arcane_Encyclopedia,Brawl-...","Pelakka_Wurm,Druid_of_the_Cowl,Sift,Centaur_Co...","Djinn_of_Wishes,Lich's_Caress,Switcheroo,Gears..."
30001,1209160,M19,"Regal_Bloodlord,Ajani's_Welcome,Skymarch_Blood...","Lich's_Caress,Death_Baron,Star-Crowned_Stag,Ra...","Cleansing_Nova,Angel_of_the_Dawn,Pegasus_Cours...","Vine_Mare,Plague_Mare,Centaur_Courser,Suspicio...","Sarkhan,_Fireblood,Druid_of_the_Cowl,Electrify...","Lich's_Caress,Vampire_Sovereign,Child_of_Night...","Angel_of_the_Dawn,Militia_Bugler,Gargoyle_Sent...","Lightning_Strike,Sparktongue_Dragon,Dark-Dwell..."
30002,1267217,M19,"Djinn_of_Wishes,Essence_Scatter,Enigma_Drake,E...","Vampire_Sovereign,Ravenous_Harpy,Abnormal_Endu...","Goreclaw,_Terror_of_Qal_Sisma,Lich's_Caress,Ar...","Switcheroo,Exclusion_Mage,Fell_Specter,Skyscan...","Cleansing_Nova,Volcanic_Dragon,Militia_Bugler,...","Bone_Dragon,Lich's_Caress,Salvager_of_Secrets,...","Vine_Mare,Electrify,Electrify,Suspicious_Bookc...","Herald_of_Faith,Aviation_Pioneer,Herald_of_Fai..."
30003,1296242,M19,"Djinn_of_Wishes,Exclusion_Mage,Divination,Esse...","Lich's_Caress,Regal_Bloodlord,Regal_Bloodlord,...","Luminous_Bonds,Vigilant_Baloth,Declare_Dominan...","Lena,_Selfless_Champion,Gallant_Cavalry,Star-C...","Militia_Bugler,Dwarven_Priest,Switcheroo,Rabid...","Palladia-Mors,_the_Ruiner,Shock,Fire_Elemental...","Banefire,Rabid_Bite,Liliana's_Contract,Onakke_...","Hungering_Hydra,Electrify,Luminous_Bonds,Satyr..."
30004,1319830,M19,"Bone_Dragon,Chaos_Wand,Regal_Bloodlord,Isolate...","Remorseful_Cleric,Volcanic_Dragon,Shock,Skysca...","Sarkhan's_Unsealing,Strangling_Spores,Boggart_...","Knightly_Valor,Knightly_Valor,Aethershield_Art...","Plague_Mare,Volcanic_Dragon,Departed_Deckhand,...","Druid_of_the_Cowl,Poison-Tip_Archer,Ravenous_H...","Sleep,Meteor_Golem,Scholar_of_Stars,Salvager_o...","Lightning_Strike,Pegasus_Courser,Suspicious_Bo..."


### Combining all above functions, and looping through each row of the dataframe, only outputting onehot

#### If you want to also generate dicts of string names for errorproof, you can do so by removing full line comments

In [ ]:
def prepare_data(dataframe, start_index):
    
    #preloop prep
    prefix_str = "player_"
    list_of_players = []
    for i in range(1, 9):
        list_of_players.append(prefix_str + str(i))
    # we now have a list of 8 player names names
    
#     dataframe_pact_dict = {}
    dataframe_pact_dict_onehot = {}
#     dataframe_packs_shown_player = {}
    dataframe_packs_shown_player_onehot = {}
#     dataframe_pool_at_pick = {}
    dataframe_pool_at_pick_onehot = {}
    dataframe_pick_onehot = {}
    
    for i in range(0, len(dataframe["player_1"])): #for each draft in the dataset
        
        player_split_draft = split_draft_text_by_player(dataframe, list_of_players, i + start_index) #take the text draft and split by player

        seperated_pools = regex_to_list_picks(player_split_draft) #use regex to make array of picks
        
        pack_seperated_draft = seperate_pool_into_packs(seperated_pools) #split each players pool into groups of 15
        
        pack_dict, pack_dict_onehot = generate_packs_opened_dict(pack_seperated_draft) #generate dict of each players packs opened
        
#         dataframe_pact_dict["draft_" + str(i)] = pack_dict #save pack dict to dataframe dict

        dataframe_pact_dict_onehot["draft_" + str(i)] = pack_dict_onehot #save pack onehot to datafram dict
        
        feeder_array = generate_feeder_array(pack_dict) #generate feeder array used for future steps
        
        packs_shown_player, pool_at_pick, packs_shown_player_onehot, pool_at_pick_onehot, pick_onehot = create_all_picks_seen(feeder_array, pack_seperated_draft, seperated_pools, 0)
        
#         dataframe_packs_shown_player["draft_" + str(i)] = packs_shown_player

#         dataframe_pool_at_pick["draft_" + str(i)] = pool_at_pick

        dataframe_packs_shown_player_onehot["draft_" + str(i)] = packs_shown_player_onehot
    
        dataframe_pool_at_pick_onehot["draft_" + str(i)] = pool_at_pick_onehot
        
        dataframe_pick_onehot["draft_" + str(i)] = pick_onehot
        
        # above line generates four dicts, then saves them to dataframe dicts
        
    return dataframe_pact_dict_onehot, dataframe_packs_shown_player_onehot, dataframe_pool_at_pick_onehot, dataframe_pick_onehot #dataframe_pact_dict, dataframe_packs_shown_player, dataframe_pool_at_pick,

## Generating the formatted Data

In [ ]:
dataframe_pact_dict_onehot_train, dataframe_packs_shown_player_onehot_train, dataframe_pool_at_pick_onehot_train, dataframe_pick_onehot_train = prepare_data(drafts_train_export, drafts_train_start)

In [ ]:
dataframe_pact_dict_onehot_test, dataframe_packs_shown_player_onehot_test, dataframe_pool_at_pick_onehot_test, dataframe_pick_onehot_test = prepare_data(drafts_test_export, drafts_test_start)

### Example output

### Testing things are working using find_contense function

#### All picks should be in corresponding packs.  All pools should contain all previous picks.  We can make sure things are working correctly by examing a few test cases and making sure all of these conditions hold

###### Pack

In [ ]:
find_contense(dataframe_packs_shown_player_onehot_test["draft_10"]["player_1_pack_1_pick_6"])

['Mountain_4',
 'Reclamation_Sage',
 'Dwarven_Priest',
 'Abnormal_Endurance',
 'Gearsmith_Guardian',
 'Manalith',
 'Oakenform',
 'Trumpet_Blast',
 'Root_Snare',
 'Boggart_Brute']

###### Pick

In [ ]:
find_contense(dataframe_pick_onehot_test["draft_855"]["player_1_pack_1_pick_6"])

['Giant_Spider']

###### Pool (note, add 1 to pick because pool only contains card once it is picked (it is included in the following pick)

In [ ]:
find_contense(dataframe_pool_at_pick_onehot_test["draft_855"]["player_1_pack_1_pick_7"])

['Gigantosaurus',
 "Magistrate's_Scepter",
 'Mirror_Image',
 'Giant_Spider',
 'Recollect',
 'Millstone']

## Exporting the output to a CSV

In [ ]:
test_dataset = {}
test_dataset["packs"] = dataframe_packs_shown_player_onehot_test
test_dataset["pools"] = dataframe_pool_at_pick_onehot_test
test_dataset["picks"] = dataframe_pick_onehot_test

In [ ]:
train_dataset = {}
train_dataset["packs"] = dataframe_packs_shown_player_onehot_train
train_dataset["pools"] = dataframe_pool_at_pick_onehot_train
train_dataset["picks"] = dataframe_pick_onehot_train

In [ ]:
card_indexes = {}
card_indexes["index"] = card_index
card_indexes["inverted"] = card_index_inverted

In [ ]:
# test_data_save = open("/content/gdrive/MyDrive/Colab Notebooks/test_data_save_batch_two", "wb")
# pickle.dump(test_dataset, test_data_save)
# test_data_save.close()

In [ ]:
# train_data_save = open("/content/gdrive/MyDrive/Colab Notebooks/train_data_save_batch_two", "wb")
# pickle.dump(train_dataset, train_data_save)
# train_data_save.close()

In [ ]:
# index_data_save = open("/content/gdrive/MyDrive/Colab Notebooks/index_data_save", "wb")
# pickle.dump(card_indexes, index_data_save)
# index_data_save.close()